In [ ]:
# ===================================
# Sentiment & Emotion Detection App
# ===================================
# This project detects both sentiment (positive, negative, neutral)
# and emotion (happy, sad, angry, etc.) from:
#   1. Text Input
#   2. Audio Input (via Speech-to-Text)
#
# It uses:
#   - Hugging Face Transformers for NLP
#   - OpenAI Whisper for Speech Recognition
#   - Gradio for a web-based UI
# ===================================

print("📢 Welcome to the Sentiment & Emotion Detection Project!")


📢 Welcome to the Sentiment & Emotion Detection Project!


In [ ]:
# Install all required packages
!pip install gradio transformers torch librosa git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-6vfuhn2f
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-6vfuhn2f
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━

In [ ]:
# Import necessary libraries
import gradio as gr
from transformers import pipeline
import whisper


In [ ]:
# ================================
# Loading Pre-trained Models
# ================================
print("🔄 Loading models... This may take a minute.")

# Sentiment analysis model
sentiment_model = pipeline("sentiment-analysis")

# Emotion detection model
emotion_model = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base"
)

# Whisper speech-to-text model
whisper_model = whisper.load_model("small")  # "small" for speed
print("✅ Models loaded successfully!")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


🔄 Loading models... This may take a minute.


Device set to use cpu
Device set to use cpu


✅ Models loaded successfully!


In [ ]:
# ================================
# Loading Pre-trained Models
# ================================
print("🔄 Loading models... This may take a minute.")

# Determine the device to use
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

try:
    # Sentiment analysis model
    sentiment_model = pipeline("sentiment-analysis", device=device)
    print("✅ Sentiment model loaded successfully!")
except Exception as e:
    print(f"❌ Failed to load sentiment model: {e}")
    sentiment_model = None

try:
    # Emotion detection model
    emotion_model = pipeline(
        "text-classification",
        model="j-hartmann/emotion-english-distilroberta-base",
        device=device
    )
    print("✅ Emotion detection model loaded successfully!")
except Exception as e:
    print(f"❌ Failed to load emotion detection model: {e}")
    emotion_model = None

try:
    # Whisper speech-to-text model
    whisper_model = whisper.load_model("small", device=device)  # "small" for speed
    print("✅ Whisper model loaded successfully!")
except Exception as e:
    print(f"❌ Failed to load Whisper model: {e}")
    whisper_model = None


# Function to analyze text
def analyze_text(text):
    if not sentiment_model or not emotion_model:
        return "Models not loaded. Please check the logs."
    sentiment = sentiment_model(text)[0]
    emotion = emotion_model(text)[0]
    return (
        text,
        sentiment['label'], round(sentiment['score'], 2),
        emotion['label'], round(emotion['score'], 2)
    )

# Function to analyze audio
def analyze_audio(audio_file):
    if not whisper_model or not sentiment_model or not emotion_model:
         return "Models not loaded. Please check the logs."
    # Transcribe audio to text
    result = whisper_model.transcribe(audio_file)
    text = result["text"]
    return analyze_text(text)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


🔄 Loading models... This may take a minute.
Using device: cpu


Device set to use cpu


✅ Sentiment model loaded successfully!


Device set to use cpu


✅ Emotion detection model loaded successfully!
✅ Whisper model loaded successfully!


In [ ]:
# ================================
# Creating the Gradio Interface
# ================================
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🎯 Sentiment & Emotion Detector
        **Analyze text or audio to detect sentiment and emotion.**
        Powered by **Hugging Face Transformers** and **OpenAI Whisper**.
        Supports **multilingual input** (audio auto-detected).
        """
    )

    with gr.Tab("📝 Text Input"):
        text_input = gr.Textbox(lines=3, placeholder="Type your text here...")
        sentiment_label = gr.Label(label="Sentiment")
        sentiment_conf = gr.Slider(0, 1, label="Sentiment Confidence", interactive=False)
        emotion_label = gr.Label(label="Emotion")
        emotion_conf = gr.Slider(0, 1, label="Emotion Confidence", interactive=False)
        text_button = gr.Button("Analyze Text")

        text_button.click(
            analyze_text,
            inputs=text_input,
            outputs=[text_input, sentiment_label, sentiment_conf, emotion_label, emotion_conf]
        )

    with gr.Tab("🎤 Audio Input"):
        audio_input = gr.Audio(sources=["upload", "microphone"], type="filepath", label="Upload or Record Audio")
        sentiment_label_a = gr.Label(label="Sentiment")
        sentiment_conf_a = gr.Slider(0, 1, label="Sentiment Confidence", interactive=False)
        emotion_label_a = gr.Label(label="Emotion")
        emotion_conf_a = gr.Slider(0, 1, label="Emotion Confidence", interactive=False)
        audio_button = gr.Button("Analyze Audio")

        audio_button.click(
            analyze_audio,
            inputs=audio_input,
            outputs=[audio_input, sentiment_label_a, sentiment_conf_a, emotion_label_a, emotion_conf_a]
        )


In [ ]:
# ================================
# Launch Gradio App in Browser
# ================================
demo.launch(share=True)  # share=True for public browser link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://657f4a2a686d413de2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Test Text Analysis
print(analyze_text("I am very happy today!"))

# Test Audio Analysis (upload a file manually first)
# from google.colab import files
# uploaded = files.upload()
# for fn in uploaded.keys():
#     print(analyze_audio(fn))


('I am very happy today!', 'POSITIVE', 1.0, 'joy', 0.97)


In [ ]:
# Install dependencies
!pip install gradio transformers torch librosa git+https://github.com/openai/whisper.git

import gradio as gr
from transformers import pipeline
import whisper

# Load models
sentiment_model = pipeline("sentiment-analysis")
emotion_model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")
whisper_model = whisper.load_model("small")

# Text analysis function
def analyze_text(text):
    sentiment = sentiment_model(text)[0]
    emotion = emotion_model(text)[0]
    return f"Sentiment: {sentiment['label']} ({sentiment['score']:.2f})\nEmotion: {emotion['label']} ({emotion['score']:.2f})"

# Audio analysis function
def analyze_audio(audio_file):
    result = whisper_model.transcribe(audio_file)
    text = result["text"]
    return analyze_text(text)

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Sentiment & Emotion Detector\n### Works with Text or Audio")

    with gr.Tab("Text Input"):
        text_input = gr.Textbox(label="Enter Text")
        text_output = gr.Textbox(label="Result")
        text_button = gr.Button("Analyze Text")
        text_button.click(analyze_text, inputs=text_input, outputs=text_output)

    with gr.Tab("Audio Input"):
        audio_input = gr.Audio(sources=["upload", "microphone"], type="filepath", label="Upload or Record Audio")
        audio_output = gr.Textbox(label="Result")
        audio_button = gr.Button("Analyze Audio")
        audio_button.click(analyze_audio, inputs=audio_input, outputs=audio_output)

demo.launch(share=True)  # share=True gives you public link


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-g39k6bnc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-g39k6bnc
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c859da381f5199bf92.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
